In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_arch_list())
print(torch.__path__)
(torch.__version__)

True
['sm_50', 'sm_60', 'sm_70', 'sm_75', 'sm_80', 'sm_86', 'sm_90']
['/usr/local/lib/python3.12/dist-packages/torch']


'2.9.0+cu126'

In [3]:
# Параметры обучения
num_epoches = 21
batch_size = 8

trocr_model = "/content/drive/MyDrive/TrOCR/models/model_orig_fine_tunes"

# Датасет
## Загрузка датасета

In [4]:
!pip install -q transformers evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 123.3 MB/s eta 0:00:00


In [5]:
import pandas as pd

train_val_df = pd.read_csv(
    "/content/drive/MyDrive/TrOCR/easy_fields_dataset/train.tsv",
   sep="\t",
   header=None,
   names=["file_name", "text"],
)
train_val_df.head()

,file_name,text
0,1_fuel_consumption_fact_conf_0.67_1_1258.jpg,"22,06"
1,1_fuel_consumption_fact_conf_0.70_1_1299.jpg,"55,05"
2,1_fuel_consumption_fact_conf_0.70_1_1362.jpg,8
3,1_fuel_consumption_fact_conf_0.70_1_1741.jpg,22
4,1_fuel_consumption_fact_conf_0.71_1_1425.jpg,6


In [6]:
import pandas as pd

test_df = pd.read_csv(
    "/content/drive/MyDrive/TrOCR/easy_fields_dataset/test.tsv",
   sep="\t",
   header=None,
   names=["file_name", "text"],
)
test_df.head()

,file_name,text
0,1_fuel_consumption_fact_conf_0.60_1_1677.jpg,13
1,1_fuel_consumption_fact_conf_0.62_1_1656.jpg,24
2,1_fuel_consumption_fact_conf_0.69_1_1320.jpg,55
3,1_fuel_consumption_fact_conf_0.78_1_100.jpg,32
4,1_fuel_consumption_fact_conf_0.87_1_1090.jpg,51


In [7]:
print("train:", train_val_df.isnull().sum())
print("test:", test_df.isnull().sum())

train: file_name     0
text         18
dtype: int64
test: file_name    0
text         0
dtype: int64


In [8]:
train_val_df = train_val_df.dropna()
test_df = test_df.dropna()

## Деление на train/validation, подготовка

In [9]:
from sklearn.model_selection import train_test_split

train_df, eval_df = train_test_split(train_val_df, test_size=0.2, random_state=1)

# Обнуление индекса у дата фреймов
train_df.reset_index(drop=True, inplace=True)
eval_df.reset_index(drop=True, inplace=True)

In [10]:
import torch
from torch.utils.data import Dataset
from PIL import Image


class CHDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # 1. Найти название файла и текст
        file_name = self.df["file_name"][idx]
        text = self.df["text"][idx]

        # 2. Подготовка изображения
        image = Image.open(self.root_dir + '/' + file_name).convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values

        # 3. Токенизирование текста
        labels = self.processor.tokenizer(text, padding="max_length", max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [
            label if label != self.processor.tokenizer.pad_token_id else -100
            for label in labels
        ]

        encoding = {
            "pixel_values": pixel_values.squeeze(),
            "labels": torch.tensor(labels),
        }
        return encoding

In [11]:
from transformers import TrOCRProcessor

processor = TrOCRProcessor.from_pretrained(trocr_model)

train_dataset = CHDataset(
   root_dir="/content/drive/MyDrive/TrOCR/easy_fields_dataset/train",
   df=train_df,
   processor=processor,
)
eval_dataset = CHDataset(
   root_dir="/content/drive/MyDrive/TrOCR/easy_fields_dataset/train",
   df=eval_df,
   processor=processor,
)
test_dataset = CHDataset(
   root_dir="/content/drive/MyDrive/TrOCR/easy_fields_dataset/test",
   df=test_df,
   processor=processor,
)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [12]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))
print("Number of test examples:", len(test_dataset))

Number of training examples: 344
Number of validation examples: 87
Number of test examples: 46


In [13]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

## Пример изображения и текста

In [14]:
print(train_df['file_name'][0])
print(train_dataset.root_dir)
file_name = f'{train_dataset.root_dir}/{train_df['file_name'][0]}'
print(file_name)

2_fuel_issued_conf_0.86_1_902.jpg
/content/drive/MyDrive/TrOCR/easy_fields_dataset/train
/content/drive/MyDrive/TrOCR/easy_fields_dataset/train/2_fuel_issued_conf_0.86_1_902.jpg


In [15]:
import cv2
from PIL import Image

encoding = train_dataset[0]
for k, v in encoding.items():
   print(k, v.shape)

image = Image.open(f'{train_dataset.root_dir}/{train_df['file_name'][0]}').convert("RGB")

labels = encoding['labels']
labels[labels == -100] = processor.tokenizer.pad_token_id
label_str = processor.decode(labels, skip_special_tokens=True)

print(label_str)
image

pixel_values torch.Size([3, 384, 384])
labels torch.Size([128])
48,81


# Модель
## Загрузка модели

In [16]:
from transformers import VisionEncoderDecoderModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VisionEncoderDecoderModel.from_pretrained(trocr_model)
model.to(device)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (i

In [17]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

## Обучение

In [18]:
def save_checkpoint(epoch, model, optimizer, path):
    """
    Сохраняет контрольную точку обучения.
    :param epoch: текущая эпоха
    :param model: модель pytorch
    :param optimizer: оптимизатор
    :param path: путь для сохранения чекпоинта
    """
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, path)
    print(f'Сохранен чекпоинт {epoch} эпохи')

In [19]:
def load_checkpoint(path, model, optimizer):
    """
    Восстанавливает контрольную точку обучения.
    :param path: путь к файлу чекпоинта
    :param model: модель pytorch
    :param optimizer: оптимизатор
    :return: последняя прошедшая эпоха
    """
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    return start_epoch

In [20]:
from torch.optim import AdamW
import os
from tqdm import tqdm
from transformers import TrOCRProcessor
import torch.nn.functional as F
import torch

# processor = TrOCRProcessor.from_pretrained(trocr_model)

# Настройки пути для сохранения контрольных точек
CHECKPOINT_PATH = "/content/drive/MyDrive/TrOCR/Checkpoints/trOCR_checkpoint_e20.pth"

# Функция для начала или продолжения обучения
def train_and_save_checkpoint(model, optimizer, dataloader, num_epochs, device, start_epoch=0):
    for epoch in range(start_epoch, num_epochs):
        # Train mode
        model.train()
        train_loss = 0.0
        correct_tokens = 0
        total_tokens = 0

        for batch in tqdm(dataloader):
            for k, v in batch.items():
                batch[k] = v.to(device)

            # Forward pass
            outputs = model(**batch)
            loss = outputs.loss

            # Calculate training loss
            train_loss += loss.item()

            # Backpropagation
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        avg_loss = train_loss / len(dataloader)

        # Save checkpoint at each epoch or periodically
        if epoch % 20 == 0:
          save_checkpoint(epoch, model, optimizer, CHECKPOINT_PATH)

        print(f"Epoch {epoch+1}/{num_epochs} | Loss: {avg_loss:.19f}")

# Основная логика программы
# Инициализация устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Определение модели и оптимизатора
model = VisionEncoderDecoderModel.from_pretrained(trocr_model)
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

# Определяем, начинать ли обучение с нуля или загрузить последний чекпоинт
start_epoch = 0
if os.path.exists(CHECKPOINT_PATH):
    start_epoch = load_checkpoint(CHECKPOINT_PATH, model, optimizer)
    print(f"Продолжаем обучение с эпохи {start_epoch + 1}")
else:
    print("Начинаем обучение с первой эпохи")

# Запускаем цикл обучения
train_and_save_checkpoint(model, optimizer, train_dataloader, num_epoches, device, start_epoch=start_epoch)

Начинаем обучение с первой эпохи


100%|██████████| 43/43 [05:18<00:00,  7.40s/it]


Сохранен чекпоинт 0 эпохи
Epoch 1/21 | Loss: 2.4747521769861844376


100%|██████████| 43/43 [01:57<00:00,  2.73s/it]


Epoch 2/21 | Loss: 0.3478631222144115687


...

Epoch 19/21 | Loss: 0.0003073494827244864


100%|██████████| 43/43 [01:05<00:00,  1.53s/it]


Epoch 20/21 | Loss: 0.0002167306835005103


100%|██████████| 43/43 [01:05<00:00,  1.53s/it]


Сохранен чекпоинт 20 эпохи
Epoch 21/21 | Loss: 0.0001982222256206009


In [26]:
processor.save_pretrained("/content/sample_data/models/weights")
model.save_pretrained("/content/sample_data/models/weights")

## Метрики

In [22]:
import evaluate
from typing import List
from tqdm.notebook import tqdm

cer_metric = evaluate.load("cer")
wer_metric = evaluate.load("wer")

def string_accuracy(pred_ids, label_ids) -> float:
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    pred_texts = pred_str
    gt_texts = label_str

    assert len(pred_texts) == len(gt_texts)
    correct = 0
    for pred_text, gt_text in zip(pred_texts, gt_texts):
        correct += int(pred_text == gt_text)
    return correct / len(gt_texts) * 100.


def compute_metrics(pred_ids, label_ids):
   pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
   label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
   label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

   cer = cer_metric.compute(predictions=pred_str, references=label_str)
   wer = wer_metric.compute(predictions=pred_str, references=label_str)
   accuracy = string_accuracy(pred_ids, label_ids)

   return cer, wer, accuracy


def evaluate_model(model, dataloader):
    model.eval()

    total_cer, total_wer, total_accuracy = 0., 0., 0.

    with torch.no_grad():
        for batch in tqdm(dataloader):
            outputs = model.generate(batch["pixel_values"].to(device))

            cer, wer, accuracy = compute_metrics(pred_ids=outputs, label_ids=batch["labels"])
            total_cer += cer
            total_wer += wer
            total_accuracy += accuracy

    total_cer /= len(dataloader)
    total_wer /= len(dataloader)
    total_accuracy /= len(dataloader)
    return total_cer, total_wer, total_accuracy

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [23]:
valid_cer, valid_wer, valid_accuracy = evaluate_model(
    model, eval_dataloader
)

# print("Validation CER:", valid_cer)
# print("Validation WER:", valid_wer)
print("Validation accuracy:", valid_accuracy)

  0%|          | 0/11 [00:00<?, ?it/s]

Validation accuracy: 57.629870129870135


In [24]:
test_cer, test_wer, test_accuracy = evaluate_model(
    model, test_dataloader
)

# print("Test CER:", test_cer)
# print("Test WER:", test_wer)
print("Test accuracy:", test_accuracy)

  0%|          | 0/6 [00:00<?, ?it/s]

Test accuracy: 54.166666666666664


In [25]:
image = Image.open(f'{train_dataset.root_dir}/{train_df['file_name'][6]}').convert("RGB")
pixel_values = processor(images=image, return_tensors="pt").pixel_values

generated_ids = model.generate(pixel_values.to(device))
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)
image

107
